In [217]:
%matplotlib inline
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import stats
import sqlite3
from math import sqrt
from sklearn.feature_extraction import DictVectorizer as DV
import statsmodels.api as sm

In [218]:
train=pd.read_csv('train.csv')
valid=pd.read_csv('validation.csv')

In [219]:
train["area"]=train["slotheight"]*train["slotwidth"]
valid["area"]=valid["slotheight"]*valid["slotwidth"]

In [220]:
from sklearn import preprocessing
import statsmodels.api as sm

label_encoder = preprocessing.LabelEncoder()
train['hour']=label_encoder.fit_transform(train['hour'])
train['weekday']=label_encoder.fit_transform(train['weekday'])
train['advertiser']=label_encoder.fit_transform(train['advertiser'])
train['area']=label_encoder.fit_transform(train['area'])
train['adexchange']=label_encoder.fit_transform(train['adexchange'])
train['slotprice']=label_encoder.fit_transform(train['slotprice'])
train['slotformat']=label_encoder.fit_transform(train['slotformat'])
train['slotvisibility']=label_encoder.fit_transform(train['slotvisibility'])

features =['weekday', 'hour','advertiser','area','adexchange','slotprice','slotformat','slotvisibility']

valid['hour']=label_encoder.fit_transform(valid['hour'])
valid['weekday']=label_encoder.fit_transform(valid['weekday'])
valid['advertiser']=label_encoder.fit_transform(valid['advertiser'])
valid['area']=label_encoder.fit_transform(valid['area'])
valid['adexchange']=label_encoder.fit_transform(valid['adexchange'])
valid['slotprice']=label_encoder.fit_transform(valid['slotprice'])
valid['slotformat']=label_encoder.fit_transform(valid['slotformat'])
valid['slotvisibility']=label_encoder.fit_transform(valid['slotvisibility'])

In [221]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier

In [150]:
x_train=train[features]
y_train=train['click']
x_test=valid[features]
params = {'n_estimators': 250, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.05, 'loss': 'deviance'}
clf = ensemble.GradientBoostingClassifier(**params)

clf.fit(x_train, y_train)

y_pred = clf.predict_proba(x_test)
y_pred

array([[  9.98989495e-01,   1.01050509e-03],
       [  9.93105198e-01,   6.89480171e-03],
       [  9.98859870e-01,   1.14012961e-03],
       ..., 
       [  9.99476845e-01,   5.23154576e-04],
       [  9.99757274e-01,   2.42725763e-04],
       [  9.98756041e-01,   1.24395867e-03]])

In [222]:
pre_1=[]
for a in y_pred:
    pre_1.append(a[1])

In [223]:
y_pred1=valid['click']

In [224]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(y_pred1, pre_1, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.027435186461778945

In [225]:
import numpy as np
from sklearn.metrics import roc_auc_score
auc=roc_auc_score(y_pred1, pre_1)
auc

0.69404273886925638

In [226]:
avg_CTR=sum(train['click'])/len(train)
avg_CTR

0.0007539649884458758

In [227]:
base_bid=75

In [228]:
L1=valid['bidprice'].values/1000
L2=valid['payprice'].values/1000

In [229]:
L=[]
for i in range(0,299749):
    p_bid=base_bid*(pre_1[i])/avg_CTR/1000
    if p_bid>=(L2[i]):
        L.append(L2[i])
    else:
        L.append(0)

In [231]:
new_bid = pd.DataFrame(L,columns=['pre_bid'])

In [232]:
frames=[valid,new_bid]
new_valid=pd.concat(frames,axis=1)

In [233]:
new_valid=new_valid[new_valid.pre_bid!=0]


In [234]:
ctr_cons= sum(new_valid.click)/len(new_valid)
click_cons = sum(new_valid.click)
cost=sum(new_valid["payprice"])/1000
avg_CPM = sum(new_valid.payprice)/1000/(len(new_valid.logtype)/1000)
eCPC = sum(new_valid.payprice)/1000/sum(new_valid.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000860253533922
125
6728.313
46.3044402846
53.826504


In [235]:
cumsum_cost = np.cumsum(new_valid.payprice)
new_valid["cumsum_cost"] = cumsum_cost
new_valid1 = new_valid.loc[(cumsum_cost < 3125000)]
new_valid2 = new_valid.loc[(cumsum_cost < 6250000)]



In [236]:
ctr_cons= sum(new_valid1.click)/len(new_valid1)
click_cons = sum(new_valid1.click)
cost=sum(new_valid1["payprice"])/1000
avg_CPM = sum(new_valid1.payprice)/1000/(len(new_valid1.logtype)/1000)
eCPC = sum(new_valid1.payprice)/1000/sum(new_valid1.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000929189834958
63
3124.984
46.0905296382
49.6029206349


In [237]:
ctr_cons= sum(new_valid2.click)/len(new_valid2)
click_cons = sum(new_valid2.click)
cost=sum(new_valid2["payprice"])/1000
avg_CPM = sum(new_valid2.payprice)/1000/(len(new_valid2.logtype)/1000)
eCPC = sum(new_valid2.payprice)/1000/sum(new_valid2.click)
print(ctr_cons)
print(click_cons)
print(cost)
print(avg_CPM)
print(eCPC)

0.000843457287027
114
6249.996
46.2421462289
54.8245263158
